<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/Limpieza_SO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Limipieza SOCIOS

In [1]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.9 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt

In [14]:
# CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
CU_SO = pd.read_excel('/content/CU_SO.xlsx')
TO_SO = pd.read_excel('/content/TO_SO.xlsx')
# AB_SO = pd.read_excel('/content/AB_SO.xlsx')
GU_SO = pd.read_excel('/content/GU_SO.xlsx')
CR_SO = pd.read_excel('/content/CR_SO.xlsx')

In [23]:
GU_SO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Género     130 non-null    object 
 1   Fecha nac  130 non-null    object 
 2   Localidad  130 non-null    object 
 3   CP         130 non-null    int64  
 4   Disc       130 non-null    object 
 5   %          24 non-null     float64
 6   Prót aud   88 non-null     object 
 7   Tipo       30 non-null     object 
 8   Mod com    33 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 9.3+ KB


In [44]:
# Definir la función que realiza las transformaciones
def transform_df(df):
    col_excluded = ['FECHA NACIMIENTO', 'FECHA DE NACIMIENTO', 'FECHA NAC']

    # Convertir todas las columnas de tipo object a mayúsculas, excepto las especificadas
    for column in df.columns:
        if df[column].dtype == 'object' and column not in col_excluded:
            df[column] = df[column].apply(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

    # Ahora, aplicamos unidecode a los nombres de las columnas
    df.columns = [unidecode(col.upper()) for col in df.columns]

    return df

def mapping_names(df):
    # Diccionario con los mapeos deseados
    columns_map = {
        'CODIGO POSTAL': 'CP',
        'MUNICIPIO': 'LOCALIDAD',
        'FECHA DE SOCIO/A': 'FECHA ALTA SOCIO',
        'FECHA NAC': 'FECHA NACIMIENTO',
        'F NAC': 'FECHA NACIMIENTO',
        'PROT AUD': 'PROTESIS',
    }

    # Crear un nuevo diccionario para los nombres de columnas
    rename_columns = {}

    # Iterar sobre las columnas y aplicar el mapeo
    for col in df.columns:
        norm_col = columns_map.get(col, col)
        rename_columns[col] = norm_col

    # Renombrar las columnas del DataFrame
    df.rename(columns=rename_columns, inplace=True)
    return df

def CP_year(df):
    if 'FECHA NACIMIENTO' in df.columns:
        if not pd.api.types.is_datetime64_any_dtype(df['FECHA NACIMIENTO']):
            df['FECHA NACIMIENTO'] = pd.to_datetime(df['FECHA NACIMIENTO'], errors='coerce')

        df['FECHA NACIMIENTO'] = df['FECHA NACIMIENTO'].apply(lambda x: np.nan if str(x).isdigit() or x == '(ADULTO)' else x)
        df['YEAR NACIMIENTO'] = df['FECHA NACIMIENTO'].dt.year

    if 'CP' in df.columns:
        # Convertir a numérico, dejando como NaN los valores que no sean convertibles
        df['CP'] = pd.to_numeric(df['CP'], errors='coerce').astype('Int64')

        df = df.dropna(subset=['CP', 'LOCALIDAD'], how='all')
        CP_loc_filter = df['CP'].isnull() & df['LOCALIDAD'].isnull()
        df = df[~CP_loc_filter]

    elif 'LOCALIDAD' in df.columns:
        df = df.dropna(subset=['LOCALIDAD'], how='all')
        loc_filter = df['LOCALIDAD'].isnull()
        df = df[~loc_filter]

    # Limpiar los datos eliminando contenido entre paréntesis
    df['LOCALIDAD'] = df['LOCALIDAD'].str.replace(r'\s*\([^)]*\)', '', regex=True)
    return df


def keep_columns(df):
    desired_columns = ['FECHA NACIMIENTO', 'GENERO', 'LOCALIDAD', 'CP', 'PROVINCIA', 'YEAR NACIMIENTO', 'FECHA ALTA SOCIO']
    # Filtrar el DataFrame para incluir solo las columnas que existen en el DataFrame
    filtered_df = df[df.columns.intersection(desired_columns)]
    return filtered_df

In [45]:
# AB_SO = transform_df(AB_SO)
# AB_SO = mapping_names(AB_SO)
# AB_SO = CP_year(AB_SO)
# AB_SO = keep_columns(AB_SO)

CU_SO = transform_df(CU_SO)
CU_SO = mapping_names(CU_SO)
CU_SO = CP_year(CU_SO)
CU_SO = keep_columns(CU_SO)

CR_SO = transform_df(CR_SO)
CR_SO = mapping_names(CR_SO)
CR_SO = CP_year(CR_SO)
CR_SO = keep_columns(CR_SO)

GU_SO = transform_df(GU_SO)
GU_SO = mapping_names(GU_SO)
GU_SO = CP_year(GU_SO)
GU_SO = keep_columns(GU_SO)

TO_SO = transform_df(TO_SO)
TO_SO = mapping_names(TO_SO)
TO_SO = CP_year(TO_SO)
TO_SO = keep_columns(TO_SO)

In [47]:
CLM_SO = pd.concat([CU_SO, CR_SO, GU_SO, TO_SO])

In [50]:
CLM_SO.info()

<class 'pandas.core.frame.DataFrame'>
Index: 791 entries, 0 to 490
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   GENERO            309 non-null    object        
 1   CP                306 non-null    Int64         
 2   LOCALIDAD         791 non-null    object        
 3   PROVINCIA         61 non-null     object        
 4   FECHA ALTA SOCIO  58 non-null     datetime64[ns]
 5   FECHA NACIMIENTO  241 non-null    datetime64[ns]
 6   YEAR NACIMIENTO   241 non-null    float64       
dtypes: Int64(1), datetime64[ns](2), float64(1), object(3)
memory usage: 50.2+ KB
